In [1]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from ase.io import read
from ase.visualize import view
from ase.units import create_units

# Test

In [2]:
atoms = read("dump/virial_0.dump")
# view(atoms)

In [3]:
cell = atoms.cell
positions = atoms.positions
forces = atoms.get_forces()

In [4]:
h = 3.4  # van der Waals thickness
V = cell[0, 0] * cell[1, 1] * h
positions.T @ forces / V

array([[-0.01115461,  0.03122681, -0.09123435],
       [-0.01360232,  0.05549371,  0.00761054],
       [-0.03815746, -0.04821287, -0.13903661]])

# Real calculation

How to compute virial stress tensor:
\begin{equation}
    \sigma = \frac{1}{N_{\tau}} \sum_{\tau = 1}^{N_{\tau}} \left( \frac{1}{V} \sum_{\alpha = 1}^N \mathbf{r}_{\tau}^{\alpha} \otimes \mathbf{f}_{\tau}^{\alpha} \right).
\end{equation}
The inner sum is done over the atoms in a single snapshot.
The outer sum is done over all the snapshots.

In [5]:
u = create_units("2018")
files = sorted(glob("dump/*.dump"))[1:]
# files

In [6]:
h = 3.4  # van der Waals thickness
sigmas = np.zeros((len(files), 3, 3))

for ii, ff in tqdm(enumerate(files), total=len(files)):
    # Read
    atoms = read(ff)
    # Extract
    cell = atoms.cell
    positions = atoms.positions
    forces = atoms.get_forces()
    # Compute
    V = cell[0, 0] * cell[1, 1] * h
    sigmas[ii] = positions.T @ forces / V

100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 209.31it/s]


In [7]:
sigma = np.mean(sigmas, axis=0)
sigma /= u.GPa  # Convert the unit to GPa
print(sigma)

[[-0.38330489  0.12335871 -0.68875897]
 [ 0.19055816 -0.6784517  -0.82537462]
 [ 0.41586569 -0.02686237 -1.86507366]]


In [8]:
# Lattice constants to try
alist = np.linspace(0.95, 1.05, 11) * 2.466
alist

array([2.3427 , 2.36736, 2.39202, 2.41668, 2.44134, 2.466  , 2.49066,
       2.51532, 2.53998, 2.56464, 2.5893 ])